# 결측치 처리방식에 따라 달라지는 분류기의 성능 확인


## 1. 결측이 발생한 행, 열정보를 삭제하는 방법
> - dropna를 활용한 결측치가 존재하는 행 또는 열정보 제거
>   - df.dropna(axis=0) # axis=0 열, axis=1 행

## 2. 특정 값을 활용한 결측치 처리 방법
2-1. fillna(값)를 통한 특정 값으로 채우기
> - 0으로 채우기: df.fillna(0)
> - 평균 : df.fillna(df.mean())
> - 중앙값 : df.fillna(df.median())
> - 최빈값 : df.fillna(df.mode())
> - 이전행의 값으로 채우기: df.fillna(method='ffill')
> - 바로 다음행의 값으로 채우기: df.fillna(method='bfill')

2-2. 그룹연산(groupby)으로 분류된 각 그룹 단위의 평균값을 활용하여 채우기
> ```
> fill_mean_grp = lambda g: g.fillna(g.mean())
> df = df.groupby('그룹조건컬럼명').apply(fill_mean_grp)
> ```

2-3. 간단한 선형비례를 이용하여 대체하는 방법
> - df = df.interpolate(method='values')


## 3. 다른 알고리즘를 활용한 결측치 처리 방법
- KNN을 활용하여 유사한 패턴을 보이는 데이터의 값을 참고하여 대체 <br>
  이때 column 및 index값이 숫자로 변경됨
> ```
> imputer = KNNImputer(n_neighbors=3) # n_neighbors: 결측치 처리에 참고할 이웃값 수
> knn_data = pd.DataFrame(imputer.fit_transform(df))
> ```



# 1. 결측이 발생한 행 또는 열정보 제거: dropna(axis=축방향)
- axis=0 # 열 제거
- axis=1 # 행 제거


In [ ]:
import pandas as pd
import numpy as np
import warnings

# 경고 메시지 출력 표기 생략
warnings.filterwarnings('ignore')

# 데이터 불러오기
# 파일 경로 = "https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/csv/married_full.csv"
married_dataset = pd.read_csv("https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/csv/married_full.csv")

#한번에 생략없이 출력하고 싶은 컬럼 수 설정
#pd.options.display.max_columns = 25

married_dataset

,gender,age,age_partner,importance_same_religion,pref_of_partner_attractive,pref_of_partner_sincere,pref_of_partner_intelligence,pref_of_partner_funny,pref_of_partner_ambitious,pref_of_partner_shared_interests,...,my_eval_sincere,my_eval_intelligence,my_eval_funny,my_eval_ambition,my_eval_shared_interests,interests_correlate,expected_happy_with_couple_match,how_much_i_liked,guess_prob_liked,married
0,female,21.0,27.0,4.0,35.0,20.0,20.0,20.0,0.0,5.0,...,9.0,7.0,7.0,6.0,5.0,0.14,3.0,7.0,6.0,0
1,female,21.0,22.0,4.0,60.0,0.0,0.0,40.0,0.0,0.0,...,8.0,7.0,8.0,5.0,6.0,0.54,3.0,7.0,5.0,0
2,female,21.0,22.0,4.0,19.0,18.0,19.0,18.0,14.0,12.0,...,8.0,9.0,8.0,5.0,7.0,0.16,3.0,7.0,5.0,1
3,female,21.0,23.0,4.0,30.0,5.0,15.0,40.0,5.0,5.0,...,6.0,8.0,7.0,6.0,8.0,0.61,3.0,7.0,6.0,1
4,female,21.0,24.0,4.0,30.0,10.0,20.0,10.0,10.0,20.0,...,6.0,7.0,7.0,6.0,6.0,0.21,3.0,6.0,6.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8357,male,25.0,26.0,1.0,10.0,10.0,30.0,20.0,10.0,15.0,...,5.0,5.0,5.0,NaN,NaN,0.64,10.0,2.0,5.0,0
8358,male,25.0,24.0,1.0,50.0,20.0,10.0,5.0,10.0,5.0,...,6.0,8.0,4.0,4.0,NaN,0.71,10.0,4.0,4.0,0
8359,male,25.0,29.0,1.0,40.0,10.0,30.0,10.0,10.0,NaN,...,7.0,8.0,8.0,8.0,NaN,-0.46,10.0,6.0,5.0,0
8360,male,25.0,22.0,1.0,10.0,25.0,25.0,10.0,10.0,20.0,...,6.0,5.0,4.0,NaN,5.0,0.62,10.0,5.0,5.0,0


In [ ]:
# 데이터셋 결측치 수 확인
married_dataset.isnull().sum()

gender                                0
age                                  72
age_partner                         102
importance_same_religion             62
pref_of_partner_attractive           83
pref_of_partner_sincere              83
pref_of_partner_intelligence         83
pref_of_partner_funny                91
pref_of_partner_ambitious            98
pref_of_partner_shared_interests    120
eval_by_partner_attractive          204
eval_by_partner_sincere             269
eval_by_partner_intelligence        291
eval_by_partner_funny               339
eval_by_partner_ambitous            681
eval_by_partner_shared_interests    989
my_pref_attractive                   62
my_pref_sincere                      62
my_pref_intellicence                 62
my_pref_funny                        72
my_pref_ambtition                    82
my_pref_shared_interests            104
my_eval_attractive                  198
my_eval_sincere                     271
my_eval_intelligence                286


In [ ]:
# 항목별 결측치 비율 확인
married_dataset.isna().mean()

gender                              0.000000
age                                 0.008610
age_partner                         0.012198
importance_same_religion            0.007414
pref_of_partner_attractive          0.009926
pref_of_partner_sincere             0.009926
pref_of_partner_intelligence        0.009926
pref_of_partner_funny               0.010883
pref_of_partner_ambitious           0.011720
pref_of_partner_shared_interests    0.014351
eval_by_partner_attractive          0.024396
eval_by_partner_sincere             0.032169
eval_by_partner_intelligence        0.034800
eval_by_partner_funny               0.040541
eval_by_partner_ambitous            0.081440
eval_by_partner_shared_interests    0.118273
my_pref_attractive                  0.007414
my_pref_sincere                     0.007414
my_pref_intellicence                0.007414
my_pref_funny                       0.008610
my_pref_ambtition                   0.009806
my_pref_shared_interests            0.012437
my_eval_at

In [ ]:
# 결측치를 제거하여 처리
married_dataset = married_dataset.dropna(axis=0)
married_dataset

,gender,age,age_partner,importance_same_religion,pref_of_partner_attractive,pref_of_partner_sincere,pref_of_partner_intelligence,pref_of_partner_funny,pref_of_partner_ambitious,pref_of_partner_shared_interests,...,my_eval_sincere,my_eval_intelligence,my_eval_funny,my_eval_ambition,my_eval_shared_interests,interests_correlate,expected_happy_with_couple_match,how_much_i_liked,guess_prob_liked,married
0,female,21.0,27.0,4.0,35.0,20.0,20.0,20.0,0.0,5.0,...,9.0,7.0,7.0,6.0,5.0,0.14,3.0,7.0,6.0,0
1,female,21.0,22.0,4.0,60.0,0.0,0.0,40.0,0.0,0.0,...,8.0,7.0,8.0,5.0,6.0,0.54,3.0,7.0,5.0,0
2,female,21.0,22.0,4.0,19.0,18.0,19.0,18.0,14.0,12.0,...,8.0,9.0,8.0,5.0,7.0,0.16,3.0,7.0,5.0,1
3,female,21.0,23.0,4.0,30.0,5.0,15.0,40.0,5.0,5.0,...,6.0,8.0,7.0,6.0,8.0,0.61,3.0,7.0,6.0,1
4,female,21.0,24.0,4.0,30.0,10.0,20.0,10.0,10.0,20.0,...,6.0,7.0,7.0,6.0,6.0,0.21,3.0,6.0,6.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8351,male,25.0,28.0,1.0,25.0,15.0,25.0,15.0,10.0,10.0,...,7.0,6.0,6.0,6.0,7.0,0.37,10.0,5.0,5.0,0
8352,male,25.0,26.0,1.0,10.0,30.0,20.0,15.0,15.0,10.0,...,4.0,6.0,4.0,4.0,4.0,0.27,10.0,4.0,5.0,0
8353,male,25.0,22.0,1.0,10.0,20.0,15.0,20.0,15.0,20.0,...,3.0,9.0,6.0,9.0,6.0,0.45,10.0,6.0,3.0,0
8354,male,25.0,27.0,1.0,10.0,25.0,20.0,20.0,5.0,20.0,...,7.0,9.0,8.0,7.0,8.0,0.35,10.0,6.0,6.0,0


In [ ]:
# object 타입의 문자열 변수를 숫자형으로 변환
married_dataset = pd.get_dummies(married_dataset, columns=['gender'], drop_first=True)

# 데이터셋, 독립변수와 종속변수 분리: 독립변수 -> x, 종속변수 -> y
x = married_dataset.drop(['married'], axis=1)
y = married_dataset['married']

x

,age,age_partner,importance_same_religion,pref_of_partner_attractive,pref_of_partner_sincere,pref_of_partner_intelligence,pref_of_partner_funny,pref_of_partner_ambitious,pref_of_partner_shared_interests,eval_by_partner_attractive,...,my_eval_sincere,my_eval_intelligence,my_eval_funny,my_eval_ambition,my_eval_shared_interests,interests_correlate,expected_happy_with_couple_match,how_much_i_liked,guess_prob_liked,gender_male
0,21.0,27.0,4.0,35.0,20.0,20.0,20.0,0.0,5.0,6.0,...,9.0,7.0,7.0,6.0,5.0,0.14,3.0,7.0,6.0,False
1,21.0,22.0,4.0,60.0,0.0,0.0,40.0,0.0,0.0,7.0,...,8.0,7.0,8.0,5.0,6.0,0.54,3.0,7.0,5.0,False
2,21.0,22.0,4.0,19.0,18.0,19.0,18.0,14.0,12.0,10.0,...,8.0,9.0,8.0,5.0,7.0,0.16,3.0,7.0,5.0,False
3,21.0,23.0,4.0,30.0,5.0,15.0,40.0,5.0,5.0,7.0,...,6.0,8.0,7.0,6.0,8.0,0.61,3.0,7.0,6.0,False
4,21.0,24.0,4.0,30.0,10.0,20.0,10.0,10.0,20.0,8.0,...,6.0,7.0,7.0,6.0,6.0,0.21,3.0,6.0,6.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8351,25.0,28.0,1.0,25.0,15.0,25.0,15.0,10.0,10.0,6.0,...,7.0,6.0,6.0,6.0,7.0,0.37,10.0,5.0,5.0,True
8352,25.0,26.0,1.0,10.0,30.0,20.0,15.0,15.0,10.0,9.0,...,4.0,6.0,4.0,4.0,4.0,0.27,10.0,4.0,5.0,True
8353,25.0,22.0,1.0,10.0,20.0,15.0,20.0,15.0,20.0,8.0,...,3.0,9.0,6.0,9.0,6.0,0.45,10.0,6.0,3.0,True
8354,25.0,27.0,1.0,10.0,25.0,20.0,20.0,5.0,20.0,7.0,...,7.0,9.0,8.0,7.0,8.0,0.35,10.0,6.0,6.0,True


In [ ]:
from sklearn.model_selection import train_test_split

# train_test_split를 활용한 train set, test set 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=100)

In [ ]:
# 예시로 xgboost 활용
import xgboost as xgb
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# model = xgb.XGBClassifier(n_estimators=500, max_depth=5, random_state=100)
model = xgb.XGBClassifier(random_state=100)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=100, ...)

In [ ]:
# 테스트 데이터로 예측
pred = model.predict(X_test)

accuracy_score(y_test, pred)  # accuracy

0.8376569037656904

In [ ]:
print(confusion_matrix(y_test, pred))  # confusion matrix

[[904  64]
 [130  97]]


In [ ]:
print(classification_report(y_test, pred))  # classification repor

              precision    recall  f1-score   support

           0       0.87      0.93      0.90       968
           1       0.60      0.43      0.50       227

    accuracy                           0.84      1195
   macro avg       0.74      0.68      0.70      1195
weighted avg       0.82      0.84      0.83      1195



# 2. 특정 값을 활용한 결측치 처리

## 2-1. fillna(값)를 통한 특정 값으로 채우기
> - 0으로 채우기: df.fillna(0)
> - 평균 : df.fillna(df.mean())
> - 중앙값 : df.fillna(df.median())
> - 최빈값 : df.fillna(df.mode())
> - 이전행의 값으로 채우기: df.fillna(method='ffill')
> - 바로 다음행의 값으로 채우기: df.fillna(method='bfill')

In [ ]:
import pandas as pd
import numpy as np
import warnings

# 경고 메시지 출력 표기 생략
warnings.filterwarnings('ignore')

# 데이터 불러오기
# 파일 경로 = "https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/csv/married_full.csv"
married_dataset = pd.read_csv("https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/csv/married_full.csv")

#한번에 생략없이 출력하고 싶은 컬럼 수 설정
#pd.options.display.max_columns = 25

married_dataset

,gender,age,age_partner,importance_same_religion,pref_of_partner_attractive,pref_of_partner_sincere,pref_of_partner_intelligence,pref_of_partner_funny,pref_of_partner_ambitious,pref_of_partner_shared_interests,...,my_eval_sincere,my_eval_intelligence,my_eval_funny,my_eval_ambition,my_eval_shared_interests,interests_correlate,expected_happy_with_couple_match,how_much_i_liked,guess_prob_liked,married
0,female,21.0,27.0,4.0,35.0,20.0,20.0,20.0,0.0,5.0,...,9.0,7.0,7.0,6.0,5.0,0.14,3.0,7.0,6.0,0
1,female,21.0,22.0,4.0,60.0,0.0,0.0,40.0,0.0,0.0,...,8.0,7.0,8.0,5.0,6.0,0.54,3.0,7.0,5.0,0
2,female,21.0,22.0,4.0,19.0,18.0,19.0,18.0,14.0,12.0,...,8.0,9.0,8.0,5.0,7.0,0.16,3.0,7.0,5.0,1
3,female,21.0,23.0,4.0,30.0,5.0,15.0,40.0,5.0,5.0,...,6.0,8.0,7.0,6.0,8.0,0.61,3.0,7.0,6.0,1
4,female,21.0,24.0,4.0,30.0,10.0,20.0,10.0,10.0,20.0,...,6.0,7.0,7.0,6.0,6.0,0.21,3.0,6.0,6.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8357,male,25.0,26.0,1.0,10.0,10.0,30.0,20.0,10.0,15.0,...,5.0,5.0,5.0,NaN,NaN,0.64,10.0,2.0,5.0,0
8358,male,25.0,24.0,1.0,50.0,20.0,10.0,5.0,10.0,5.0,...,6.0,8.0,4.0,4.0,NaN,0.71,10.0,4.0,4.0,0
8359,male,25.0,29.0,1.0,40.0,10.0,30.0,10.0,10.0,NaN,...,7.0,8.0,8.0,8.0,NaN,-0.46,10.0,6.0,5.0,0
8360,male,25.0,22.0,1.0,10.0,25.0,25.0,10.0,10.0,20.0,...,6.0,5.0,4.0,NaN,5.0,0.62,10.0,5.0,5.0,0


In [ ]:
# object 타입의 문자열 변수를 숫자형으로 변환
married_dataset = pd.get_dummies(married_dataset, columns=['gender'], drop_first=True)
married_dataset

,age,age_partner,importance_same_religion,pref_of_partner_attractive,pref_of_partner_sincere,pref_of_partner_intelligence,pref_of_partner_funny,pref_of_partner_ambitious,pref_of_partner_shared_interests,eval_by_partner_attractive,...,my_eval_intelligence,my_eval_funny,my_eval_ambition,my_eval_shared_interests,interests_correlate,expected_happy_with_couple_match,how_much_i_liked,guess_prob_liked,married,gender_male
0,21.0,27.0,4.0,35.0,20.0,20.0,20.0,0.0,5.0,6.0,...,7.0,7.0,6.0,5.0,0.14,3.0,7.0,6.0,0,False
1,21.0,22.0,4.0,60.0,0.0,0.0,40.0,0.0,0.0,7.0,...,7.0,8.0,5.0,6.0,0.54,3.0,7.0,5.0,0,False
2,21.0,22.0,4.0,19.0,18.0,19.0,18.0,14.0,12.0,10.0,...,9.0,8.0,5.0,7.0,0.16,3.0,7.0,5.0,1,False
3,21.0,23.0,4.0,30.0,5.0,15.0,40.0,5.0,5.0,7.0,...,8.0,7.0,6.0,8.0,0.61,3.0,7.0,6.0,1,False
4,21.0,24.0,4.0,30.0,10.0,20.0,10.0,10.0,20.0,8.0,...,7.0,7.0,6.0,6.0,0.21,3.0,6.0,6.0,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8357,25.0,26.0,1.0,10.0,10.0,30.0,20.0,10.0,15.0,10.0,...,5.0,5.0,NaN,NaN,0.64,10.0,2.0,5.0,0,True
8358,25.0,24.0,1.0,50.0,20.0,10.0,5.0,10.0,5.0,6.0,...,8.0,4.0,4.0,NaN,0.71,10.0,4.0,4.0,0,True
8359,25.0,29.0,1.0,40.0,10.0,30.0,10.0,10.0,NaN,2.0,...,8.0,8.0,8.0,NaN,-0.46,10.0,6.0,5.0,0,True
8360,25.0,22.0,1.0,10.0,25.0,25.0,10.0,10.0,20.0,5.0,...,5.0,4.0,NaN,5.0,0.62,10.0,5.0,5.0,0,True


In [ ]:
'''
0으로 채우기: df.fillna(0)
평균 : df.fillna(df.mean())
중앙값 : df.fillna(df.median())
최빈값 : df.fillna(df.mode())
이전행의 값으로 채우기: df.fillna(method='ffill')
바로 다음행의 값으로 채우기: df.fillna(method='bfill')
'''

married_dataset = married_dataset.fillna(married_dataset.median()) #변경 가능

married_dataset

,age,age_partner,importance_same_religion,pref_of_partner_attractive,pref_of_partner_sincere,pref_of_partner_intelligence,pref_of_partner_funny,pref_of_partner_ambitious,pref_of_partner_shared_interests,eval_by_partner_attractive,...,my_eval_intelligence,my_eval_funny,my_eval_ambition,my_eval_shared_interests,interests_correlate,expected_happy_with_couple_match,how_much_i_liked,guess_prob_liked,married,gender_male
0,21.0,27.0,4.0,35.0,20.0,20.0,20.0,0.0,5.00,6.0,...,7.0,7.0,6.0,5.0,0.14,3.0,7.0,6.0,0,False
1,21.0,22.0,4.0,60.0,0.0,0.0,40.0,0.0,0.00,7.0,...,7.0,8.0,5.0,6.0,0.54,3.0,7.0,5.0,0,False
2,21.0,22.0,4.0,19.0,18.0,19.0,18.0,14.0,12.00,10.0,...,9.0,8.0,5.0,7.0,0.16,3.0,7.0,5.0,1,False
3,21.0,23.0,4.0,30.0,5.0,15.0,40.0,5.0,5.00,7.0,...,8.0,7.0,6.0,8.0,0.61,3.0,7.0,6.0,1,False
4,21.0,24.0,4.0,30.0,10.0,20.0,10.0,10.0,20.00,8.0,...,7.0,7.0,6.0,6.0,0.21,3.0,6.0,6.0,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8357,25.0,26.0,1.0,10.0,10.0,30.0,20.0,10.0,15.00,10.0,...,5.0,5.0,7.0,6.0,0.64,10.0,2.0,5.0,0,True
8358,25.0,24.0,1.0,50.0,20.0,10.0,5.0,10.0,5.00,6.0,...,8.0,4.0,4.0,6.0,0.71,10.0,4.0,4.0,0,True
8359,25.0,29.0,1.0,40.0,10.0,30.0,10.0,10.0,10.64,2.0,...,8.0,8.0,8.0,6.0,-0.46,10.0,6.0,5.0,0,True
8360,25.0,22.0,1.0,10.0,25.0,25.0,10.0,10.0,20.00,5.0,...,5.0,4.0,7.0,5.0,0.62,10.0,5.0,5.0,0,True


In [ ]:
from sklearn.model_selection import train_test_split

# 데이터셋, 독립변수와 종속변수 분리: 독립변수 -> x, 종속변수 -> y
x = married_dataset.drop(['married'], axis=1)
y = married_dataset['married']

# train_test_split를 활용한 train set, test set 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=100)

In [ ]:
# 예시로 xgboost 활용
import xgboost as xgb
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# model = xgb.XGBClassifier(n_estimators=500, max_depth=5, random_state=100)
model = xgb.XGBClassifier(random_state=100)
model.fit(X_train, y_train)

# 테스트 데이터로 예측
pred = model.predict(X_test)

accuracy_score(y_test, pred)  # accuracy

0.8517632994620442

## 2-2. 그룹연산(groupby)으로 분류된 각 그룹 단위의 평균값을 활용한 결측치 처리
> ```
> fill_mean_grp = lambda g: g.fillna(g.mean())
> df = df.groupby('그룹조건컬럼명').apply(fill_mean_grp)
> ```

In [ ]:
import pandas as pd
import numpy as np
import warnings

# 경고 메시지 출력 표기 생략
warnings.filterwarnings('ignore')

# 데이터 불러오기
# 파일 경로 = "https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/csv/married_full.csv"
married_dataset = pd.read_csv("https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/csv/married_full.csv")

#한번에 생략없이 출력하고 싶은 컬럼 수 설정
#pd.options.display.max_columns = 25

married_dataset

,gender,age,age_partner,importance_same_religion,pref_of_partner_attractive,pref_of_partner_sincere,pref_of_partner_intelligence,pref_of_partner_funny,pref_of_partner_ambitious,pref_of_partner_shared_interests,...,my_eval_sincere,my_eval_intelligence,my_eval_funny,my_eval_ambition,my_eval_shared_interests,interests_correlate,expected_happy_with_couple_match,how_much_i_liked,guess_prob_liked,married
0,female,21.0,27.0,4.0,35.0,20.0,20.0,20.0,0.0,5.0,...,9.0,7.0,7.0,6.0,5.0,0.14,3.0,7.0,6.0,0
1,female,21.0,22.0,4.0,60.0,0.0,0.0,40.0,0.0,0.0,...,8.0,7.0,8.0,5.0,6.0,0.54,3.0,7.0,5.0,0
2,female,21.0,22.0,4.0,19.0,18.0,19.0,18.0,14.0,12.0,...,8.0,9.0,8.0,5.0,7.0,0.16,3.0,7.0,5.0,1
3,female,21.0,23.0,4.0,30.0,5.0,15.0,40.0,5.0,5.0,...,6.0,8.0,7.0,6.0,8.0,0.61,3.0,7.0,6.0,1
4,female,21.0,24.0,4.0,30.0,10.0,20.0,10.0,10.0,20.0,...,6.0,7.0,7.0,6.0,6.0,0.21,3.0,6.0,6.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8357,male,25.0,26.0,1.0,10.0,10.0,30.0,20.0,10.0,15.0,...,5.0,5.0,5.0,NaN,NaN,0.64,10.0,2.0,5.0,0
8358,male,25.0,24.0,1.0,50.0,20.0,10.0,5.0,10.0,5.0,...,6.0,8.0,4.0,4.0,NaN,0.71,10.0,4.0,4.0,0
8359,male,25.0,29.0,1.0,40.0,10.0,30.0,10.0,10.0,NaN,...,7.0,8.0,8.0,8.0,NaN,-0.46,10.0,6.0,5.0,0
8360,male,25.0,22.0,1.0,10.0,25.0,25.0,10.0,10.0,20.0,...,6.0,5.0,4.0,NaN,5.0,0.62,10.0,5.0,5.0,0


In [ ]:
# object 타입의 문자열 변수를 숫자형으로 변환
married_dataset = pd.get_dummies(married_dataset, columns=['gender'], drop_first=True)

In [ ]:
# 결혼 성공 여부에 따라 그룹화 하여 해당 그룹내 각 컬럼별 평균값으로 결측치 처리
fill_mean_grp = lambda g: g.fillna(g.mean())
married_grp_mean = married_dataset.groupby('married').apply(fill_mean_grp)

married_grp_mean

age  age_partner  importance_same_religion  \
married                                                     
0       0     21.0         27.0                       4.0   
        1     21.0         22.0                       4.0   
        5     21.0         25.0                       4.0   
        6     21.0         30.0                       4.0   
        7     21.0         27.0                       4.0   
...            ...          ...                       ...   
1       8341  25.0         26.0                       1.0   
        8344  25.0         23.0                       1.0   
        8346  25.0         23.0                       1.0   
        8347  25.0         27.0                       1.0   
        8355  25.0         25.0                       1.0   

              pref_of_partner_attractive  pref_of_partner_sincere  \
married                                                             
0       0                          35.00                    20.00   
        1                          60.00                     0.00   
        5                          50.00                     0.00   
        6                          35.00                    15.00   
        7                          33.33                    11.11   
...                                  ...                      ...   
1       8341                       25.00                    25.00   
        8344                       20.00                    20.00   
        8346                       20.00                    20.00   
        8347                       58.00                     5.00   
        8355                       15.00                    20.00   

              pref_of_partner_intelligence  pref_of_partner_funny  \
married                                                             
0       0                            20.00                  20.00   
        1                             0.00                  40.00   
        5                            30.00                  10.00   
        6                            25.00                  10.00   
        7                            11.11                  11.11   
...                                    ...                    ...   
1       8341                         25.00                  12.00   
        8344                         20.00                  20.00   
        8346                         20.00                  15.00   
        8347                          8.00                  10.00   
        8355                         25.00                  20.00   

              pref_of_partner_ambitious  pref_of_partner_shared_interests  \
married                                                                     
0       0                          0.00                              5.00   
        1                          0.00                              0.00   
        5                          0.00                             10.00   
        6                          5.00                             10.00   
        7                         11.11                             22.22   
...                                 ...                               ...   
1       8341                       3.00                             10.00   
        8344                       5.00                             15.00   
        8346                      10.00                             15.00   
        8347                       7.00                             12.00   
        8355                      10.00                             10.00   

              eval_by_partner_attractive  ...  my_eval_intelligence  \
married                                   ...                         
0       0                            6.0  ...                   7.0   
        1                            7.0  ...                   7.0   
        5                            7.0  ...                   7.0   
        6                            3.0  ...                   7.0 

In [ ]:
from sklearn.model_selection import train_test_split

# 데이터셋, 독립변수와 종속변수 분리: 독립변수 -> x, 종속변수 -> y
x = married_grp_mean.drop(['married'], axis=1)
y = married_grp_mean['married']

# train_test_split를 활용한 train set, test set 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=100)

In [ ]:
# 예시로 xgboost 활용
import xgboost as xgb
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# model = xgb.XGBClassifier(n_estimators=500, max_depth=5, random_state=100)
model = xgb.XGBClassifier(random_state=100)
model.fit(X_train, y_train)

# 테스트 데이터로 예측
pred = model.predict(X_test)

accuracy_score(y_test, pred)  # accuracy

0.8631201434548715

## 2-3. 간단한 선형비례를 이용하여 대체하는 방법

df = df.interpolate(method='values')

In [ ]:
import pandas as pd
import numpy as np
import warnings

# 경고 메시지 출력 표기 생략
warnings.filterwarnings('ignore')

# 데이터 불러오기
# 파일 경로 = "https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/csv/married_full.csv"
married_dataset = pd.read_csv("https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/csv/married_full.csv")

#한번에 생략없이 출력하고 싶은 컬럼 수 설정
#pd.options.display.max_columns = 25

married_dataset

,gender,age,age_partner,importance_same_religion,pref_of_partner_attractive,pref_of_partner_sincere,pref_of_partner_intelligence,pref_of_partner_funny,pref_of_partner_ambitious,pref_of_partner_shared_interests,...,my_eval_sincere,my_eval_intelligence,my_eval_funny,my_eval_ambition,my_eval_shared_interests,interests_correlate,expected_happy_with_couple_match,how_much_i_liked,guess_prob_liked,married
0,female,21.0,27.0,4.0,35.0,20.0,20.0,20.0,0.0,5.0,...,9.0,7.0,7.0,6.0,5.0,0.14,3.0,7.0,6.0,0
1,female,21.0,22.0,4.0,60.0,0.0,0.0,40.0,0.0,0.0,...,8.0,7.0,8.0,5.0,6.0,0.54,3.0,7.0,5.0,0
2,female,21.0,22.0,4.0,19.0,18.0,19.0,18.0,14.0,12.0,...,8.0,9.0,8.0,5.0,7.0,0.16,3.0,7.0,5.0,1
3,female,21.0,23.0,4.0,30.0,5.0,15.0,40.0,5.0,5.0,...,6.0,8.0,7.0,6.0,8.0,0.61,3.0,7.0,6.0,1
4,female,21.0,24.0,4.0,30.0,10.0,20.0,10.0,10.0,20.0,...,6.0,7.0,7.0,6.0,6.0,0.21,3.0,6.0,6.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8357,male,25.0,26.0,1.0,10.0,10.0,30.0,20.0,10.0,15.0,...,5.0,5.0,5.0,NaN,NaN,0.64,10.0,2.0,5.0,0
8358,male,25.0,24.0,1.0,50.0,20.0,10.0,5.0,10.0,5.0,...,6.0,8.0,4.0,4.0,NaN,0.71,10.0,4.0,4.0,0
8359,male,25.0,29.0,1.0,40.0,10.0,30.0,10.0,10.0,NaN,...,7.0,8.0,8.0,8.0,NaN,-0.46,10.0,6.0,5.0,0
8360,male,25.0,22.0,1.0,10.0,25.0,25.0,10.0,10.0,20.0,...,6.0,5.0,4.0,NaN,5.0,0.62,10.0,5.0,5.0,0


In [ ]:
# object 타입의 문자열 변수를 숫자형으로 변환
married_dataset = pd.get_dummies(married_dataset, columns=['gender'], drop_first=True)

In [ ]:
married_dataset = married_dataset.interpolate(method='values')

married_dataset

,age,age_partner,importance_same_religion,pref_of_partner_attractive,pref_of_partner_sincere,pref_of_partner_intelligence,pref_of_partner_funny,pref_of_partner_ambitious,pref_of_partner_shared_interests,eval_by_partner_attractive,...,my_eval_intelligence,my_eval_funny,my_eval_ambition,my_eval_shared_interests,interests_correlate,expected_happy_with_couple_match,how_much_i_liked,guess_prob_liked,married,gender_male
0,21.0,27.0,4.0,35.0,20.0,20.0,20.0,0.0,5.0,6.0,...,7.0,7.0,6.0,5.0,0.14,3.0,7.0,6.0,0,False
1,21.0,22.0,4.0,60.0,0.0,0.0,40.0,0.0,0.0,7.0,...,7.0,8.0,5.0,6.0,0.54,3.0,7.0,5.0,0,False
2,21.0,22.0,4.0,19.0,18.0,19.0,18.0,14.0,12.0,10.0,...,9.0,8.0,5.0,7.0,0.16,3.0,7.0,5.0,1,False
3,21.0,23.0,4.0,30.0,5.0,15.0,40.0,5.0,5.0,7.0,...,8.0,7.0,6.0,8.0,0.61,3.0,7.0,6.0,1,False
4,21.0,24.0,4.0,30.0,10.0,20.0,10.0,10.0,20.0,8.0,...,7.0,7.0,6.0,6.0,0.21,3.0,6.0,6.0,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8357,25.0,26.0,1.0,10.0,10.0,30.0,20.0,10.0,15.0,10.0,...,5.0,5.0,5.0,6.5,0.64,10.0,2.0,5.0,0,True
8358,25.0,24.0,1.0,50.0,20.0,10.0,5.0,10.0,5.0,6.0,...,8.0,4.0,4.0,6.0,0.71,10.0,4.0,4.0,0,True
8359,25.0,29.0,1.0,40.0,10.0,30.0,10.0,10.0,12.5,2.0,...,8.0,8.0,8.0,5.5,-0.46,10.0,6.0,5.0,0,True
8360,25.0,22.0,1.0,10.0,25.0,25.0,10.0,10.0,20.0,5.0,...,5.0,4.0,8.0,5.0,0.62,10.0,5.0,5.0,0,True


In [ ]:
from sklearn.model_selection import train_test_split

# 데이터셋, 독립변수와 종속변수 분리: 독립변수 -> x, 종속변수 -> y
x = married_dataset.drop(['married'], axis=1)
y = married_dataset['married']

# train_test_split를 활용한 train set, test set 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=100)

In [ ]:
# 예시로 xgboost 활용
import xgboost as xgb
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# model = xgb.XGBClassifier(n_estimators=500, max_depth=5, random_state=100)
model = xgb.XGBClassifier(random_state=100)
model.fit(X_train, y_train)

# 테스트 데이터로 예측
pred = model.predict(X_test)

accuracy_score(y_test, pred)  # accuracy

0.8529587567244471

# 3. 다른 알고리즘를 활용한 결측치 처리 방법: KNN


In [ ]:
import pandas as pd
import numpy as np
import warnings

# 경고 메시지 출력 표기 생략
warnings.filterwarnings('ignore')

# 데이터 불러오기
# 파일 경로 = "https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/csv/married_full.csv"
married_dataset = pd.read_csv("https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/csv/married_full.csv")

#한번에 생략없이 출력하고 싶은 컬럼 수 설정
#pd.options.display.max_columns = 25

married_dataset

,gender,age,age_partner,importance_same_religion,pref_of_partner_attractive,pref_of_partner_sincere,pref_of_partner_intelligence,pref_of_partner_funny,pref_of_partner_ambitious,pref_of_partner_shared_interests,...,my_eval_sincere,my_eval_intelligence,my_eval_funny,my_eval_ambition,my_eval_shared_interests,interests_correlate,expected_happy_with_couple_match,how_much_i_liked,guess_prob_liked,married
0,female,21.0,27.0,4.0,35.0,20.0,20.0,20.0,0.0,5.0,...,9.0,7.0,7.0,6.0,5.0,0.14,3.0,7.0,6.0,0
1,female,21.0,22.0,4.0,60.0,0.0,0.0,40.0,0.0,0.0,...,8.0,7.0,8.0,5.0,6.0,0.54,3.0,7.0,5.0,0
2,female,21.0,22.0,4.0,19.0,18.0,19.0,18.0,14.0,12.0,...,8.0,9.0,8.0,5.0,7.0,0.16,3.0,7.0,5.0,1
3,female,21.0,23.0,4.0,30.0,5.0,15.0,40.0,5.0,5.0,...,6.0,8.0,7.0,6.0,8.0,0.61,3.0,7.0,6.0,1
4,female,21.0,24.0,4.0,30.0,10.0,20.0,10.0,10.0,20.0,...,6.0,7.0,7.0,6.0,6.0,0.21,3.0,6.0,6.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8357,male,25.0,26.0,1.0,10.0,10.0,30.0,20.0,10.0,15.0,...,5.0,5.0,5.0,NaN,NaN,0.64,10.0,2.0,5.0,0
8358,male,25.0,24.0,1.0,50.0,20.0,10.0,5.0,10.0,5.0,...,6.0,8.0,4.0,4.0,NaN,0.71,10.0,4.0,4.0,0
8359,male,25.0,29.0,1.0,40.0,10.0,30.0,10.0,10.0,NaN,...,7.0,8.0,8.0,8.0,NaN,-0.46,10.0,6.0,5.0,0
8360,male,25.0,22.0,1.0,10.0,25.0,25.0,10.0,10.0,20.0,...,6.0,5.0,4.0,NaN,5.0,0.62,10.0,5.0,5.0,0


In [ ]:
# object 타입의 문자열 변수를 숫자형으로 변환
married_dataset = pd.get_dummies(married_dataset, columns=['gender'], drop_first=True)

In [ ]:
# 사이킷런 라이브러리의 KNNImputer 불러오기
from sklearn.impute import KNNImputer

# KNNImputer 객체 생성과 KNN알고리즘에서 중요한 n_neighbors 수(참고할 이웃값 수) 설정
imputer = KNNImputer(n_neighbors=3)

married_knn = pd.DataFrame(imputer.fit_transform(married_dataset))
married_knn

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,21.0,27.0,4.0,35.0,20.0,20.0,20.0,0.0,5.0,6.0,...,7.0,7.0,6.0,5.000000,0.14,3.0,7.0,6.0,0.0,0.0
1,21.0,22.0,4.0,60.0,0.0,0.0,40.0,0.0,0.0,7.0,...,7.0,8.0,5.0,6.000000,0.54,3.0,7.0,5.0,0.0,0.0
2,21.0,22.0,4.0,19.0,18.0,19.0,18.0,14.0,12.0,10.0,...,9.0,8.0,5.0,7.000000,0.16,3.0,7.0,5.0,1.0,0.0
3,21.0,23.0,4.0,30.0,5.0,15.0,40.0,5.0,5.0,7.0,...,8.0,7.0,6.0,8.000000,0.61,3.0,7.0,6.0,1.0,0.0
4,21.0,24.0,4.0,30.0,10.0,20.0,10.0,10.0,20.0,8.0,...,7.0,7.0,6.0,6.000000,0.21,3.0,6.0,6.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8357,25.0,26.0,1.0,10.0,10.0,30.0,20.0,10.0,15.0,10.0,...,5.0,5.0,7.0,7.333333,0.64,10.0,2.0,5.0,0.0,1.0
8358,25.0,24.0,1.0,50.0,20.0,10.0,5.0,10.0,5.0,6.0,...,8.0,4.0,4.0,7.000000,0.71,10.0,4.0,4.0,0.0,1.0
8359,25.0,29.0,1.0,40.0,10.0,30.0,10.0,10.0,5.0,2.0,...,8.0,8.0,8.0,6.333333,-0.46,10.0,6.0,5.0,0.0,1.0
8360,25.0,22.0,1.0,10.0,25.0,25.0,10.0,10.0,20.0,5.0,...,5.0,4.0,6.0,5.000000,0.62,10.0,5.0,5.0,0.0,1.0


In [ ]:
# 컬럼명이 숫자로 변환되었으므로 다시 원래의 이름으로 변환
married_knn.columns = married_dataset.columns

married_knn

,age,age_partner,importance_same_religion,pref_of_partner_attractive,pref_of_partner_sincere,pref_of_partner_intelligence,pref_of_partner_funny,pref_of_partner_ambitious,pref_of_partner_shared_interests,eval_by_partner_attractive,...,my_eval_intelligence,my_eval_funny,my_eval_ambition,my_eval_shared_interests,interests_correlate,expected_happy_with_couple_match,how_much_i_liked,guess_prob_liked,married,gender_male
0,21.0,27.0,4.0,35.0,20.0,20.0,20.0,0.0,5.0,6.0,...,7.0,7.0,6.0,5.000000,0.14,3.0,7.0,6.0,0.0,0.0
1,21.0,22.0,4.0,60.0,0.0,0.0,40.0,0.0,0.0,7.0,...,7.0,8.0,5.0,6.000000,0.54,3.0,7.0,5.0,0.0,0.0
2,21.0,22.0,4.0,19.0,18.0,19.0,18.0,14.0,12.0,10.0,...,9.0,8.0,5.0,7.000000,0.16,3.0,7.0,5.0,1.0,0.0
3,21.0,23.0,4.0,30.0,5.0,15.0,40.0,5.0,5.0,7.0,...,8.0,7.0,6.0,8.000000,0.61,3.0,7.0,6.0,1.0,0.0
4,21.0,24.0,4.0,30.0,10.0,20.0,10.0,10.0,20.0,8.0,...,7.0,7.0,6.0,6.000000,0.21,3.0,6.0,6.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8357,25.0,26.0,1.0,10.0,10.0,30.0,20.0,10.0,15.0,10.0,...,5.0,5.0,7.0,7.333333,0.64,10.0,2.0,5.0,0.0,1.0
8358,25.0,24.0,1.0,50.0,20.0,10.0,5.0,10.0,5.0,6.0,...,8.0,4.0,4.0,7.000000,0.71,10.0,4.0,4.0,0.0,1.0
8359,25.0,29.0,1.0,40.0,10.0,30.0,10.0,10.0,5.0,2.0,...,8.0,8.0,8.0,6.333333,-0.46,10.0,6.0,5.0,0.0,1.0
8360,25.0,22.0,1.0,10.0,25.0,25.0,10.0,10.0,20.0,5.0,...,5.0,4.0,6.0,5.000000,0.62,10.0,5.0,5.0,0.0,1.0


In [ ]:
from sklearn.model_selection import train_test_split

# 데이터셋, 독립변수와 종속변수 분리: 독립변수 -> x, 종속변수 -> y
x = married_knn.drop(['married'], axis=1)
y = married_knn['married']

# train set 과 test set 으로 데이터를 나누기 위해 train_test_split 활용
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=100)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
import xgboost as xgb

# model = xgb.XGBClassifier(n_estimators=500, max_depth=5, random_state=100)
model = xgb.XGBClassifier(random_state=100)
model.fit(X_train, y_train)

pred = model.predict(X_test)

print('accuracy:',accuracy_score(y_test, pred))
print('f1-score:',f1_score(y_test,pred))

accuracy: 0.8559473998804543
f1-score: 0.5170340681362726


In [ ]:
print(confusion_matrix(y_test, pred))  # confusion matrix 출력

[[1303   68]
 [ 173  129]]


In [ ]:
print(classification_report(y_test, pred))  # classification report 출력

              precision    recall  f1-score   support

         0.0       0.88      0.95      0.92      1371
         1.0       0.65      0.43      0.52       302

    accuracy                           0.86      1673
   macro avg       0.77      0.69      0.72      1673
weighted avg       0.84      0.86      0.84      1673



# 하이퍼파라미터 튜닝

- optuna


In [ ]:
import pandas as pd
import numpy as np
import warnings

# 경고 메시지 출력 표기 생략
warnings.filterwarnings('ignore')

# 데이터 불러오기
# 파일 경로 = "https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/csv/married_full.csv"
married_dataset = pd.read_csv("https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/csv/married_full.csv")

# object 타입의 문자열 변수를 숫자형으로 변환
married_dataset = pd.get_dummies(married_dataset, columns=['gender'], drop_first=True)

# 데이터셋, 독립변수와 종속변수 분리: 독립변수 -> x, 종속변수 -> y
x = married_dataset.drop(['married'], axis=1)
y = married_dataset['married']

# train set 과 test set 으로 데이터를 나누기 위해 train_test_split 함수를 불러옴
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=100)

In [ ]:
import xgboost as xgb

model = xgb.XGBClassifier(random_state=100)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=100, ...)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

pred = model.predict(X_test)

print('accuracy:',accuracy_score(y_test, pred))
print('f1-score:',f1_score(y_test,pred))

accuracy: 0.8493723849372385
f1-score: 0.48148148148148157


### XGBoost Classifier에 있는 Hyperparameter
* XGBoost API 설명자료: https://xgboost.readthedocs.io/en/stable/python/python_api.html#module-xgboost.sklearn  

* 코드에서 활용한 하이퍼 파라미터
> * objective: 모델의 학습 과제 및 학습목표(일반적으로 classifier의 경우 이진 혹은 다중 분류로 기본값이 설정되어 있음)
> * num_leaves: tree의 최대 leaf 수
> * learning_rate: 학습률
> * n_estimators: 학습에 사용할 트리 수(xgb학습과정에서 부스팅 라운드 수)
> * max_depth: 트리의 최대 깊이
> * ramdom_state: 결과 재현을 위한 시드값


In [ ]:
#현재 코랩 런타임에 Optuna 라이브러리 설치
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 7.2 MB/s eta 0:00:00


In [ ]:
import optuna

def objective(trial):
    global X_train, X_test, y_train, y_test
    xbg_trainset = xgb.DMatrix(X_train, label=y_train)
    xgb_testset = xgb.DMatrix(X_test, label=y_test)

    # 최적화할 하이퍼 파라미터 지정 및 찾아볼 값 범위 설정
    param = {
        "objective": "binary:logistic",
        'num_leaves': trial.suggest_int('num_leaves', 2, 1024, step=1, log=True),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 0.3),
        'n_estimators':  trial.suggest_int('n_estimators',100,3000 ),
        'max_depth': trial.suggest_int("max_depth", 3, 21, step=2),
        'random_state': 100,
    }

    model_train = xgb.train(param, xbg_trainset)
    preds = model_train.predict(xgb_testset)
    pred_labels = np.rint(preds)

    accuracy = accuracy_score(y_test, pred_labels)
    return accuracy

# optuna에서의 최적화 할 study 생성과 최적화 방향(지표의 값을 최대화 할 것인지)
study = optuna.create_study(direction="maximize")

# trial 횟수 설정 및 최적화 시작
study.optimize(objective, n_trials=500, timeout=600)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

[I 2024-04-29 01:15:33,914] A new study created in memory with name: no-name-ef987e7a-fd53-4e28-ab8f-a149ef2583df
[I 2024-04-29 01:15:34,109] Trial 0 finished with value: 0.8194859533771668 and parameters: {'num_leaves': 22, 'learning_rate': 0.0007320218986546918, 'n_estimators': 697, 'max_depth': 9}. Best is trial 0 with value: 0.8194859533771668.
[I 2024-04-29 01:15:34,252] Trial 1 finished with value: 0.8194859533771668 and parameters: {'num_leaves': 489, 'learning_rate': 0.0017802132915706685, 'n_estimators': 1302, 'max_depth': 7}. Best is trial 0 with value: 0.8194859533771668.
[I 2024-04-29 01:15:34,401] Trial 2 finished with value: 0.8194859533771668 and parameters: {'num_leaves': 91, 'learning_rate': 5.865982137220365e-05, 'n_estimators': 2388, 'max_depth': 7}. Best is trial 0 with value: 0.8194859533771668.
[I 2024-04-29 01:15:35,056] Trial 3 finished with value: 0.8194859533771668 and parameters: {'num_leaves': 10, 'learning_rate': 0.00012514214904140204, 'n_estimators': 2101

Number of finished trials:  500
Best trial:
  Value: 0.8511655708308428
  Params: 
    num_leaves: 72
    learning_rate: 0.2950715705379814
    n_estimators: 1222
    max_depth: 3


In [ ]:
# best trial에 대한 성능과 hyperparameter 정보 출력
print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
#plot_optimization_histor: trial 진행과정 히스토리
optuna.visualization.plot_optimization_history(study)

In [ ]:
#plot_parallel_coordinate: 하이퍼파라미터의 조합과 점수에 대한 시각화
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
#최적화 과정에서 계산된 하이퍼 파라미터 별 성능에 영향을 미친 중요도 시각화
optuna.visualization.plot_param_importances(study)

# [추가실습자료] LightGBM 알고리즘 활용하기 + optuna

In [ ]:
import pandas as pd
import numpy as np
import warnings

# 경고 메시지 출력 표기 생략
warnings.filterwarnings('ignore')

# 파일 경로 = "https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/csv/married_full.csv"
married_dataset = pd.read_csv("https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/csv/married_full.csv")

# 결측치 처리
married_dataset = married_dataset.dropna(axis=0)

# object 타입의 문자열 변수를 숫자형으로 변환
married_dataset = pd.get_dummies(married_dataset, columns=['gender'], drop_first=True)

# 데이터셋, 독립변수와 종속변수 분리: 독립변수 -> x, 종속변수 -> y
x = married_dataset.drop(['married'], axis=1)
y = married_dataset['married']

# train set 과 test set 으로 데이터를 나누기 위해 train_test_split 함수를 불러옴
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=100)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
import lightgbm as lgb

model = lgb.LGBMClassifier(random_state=100)
model.fit(X_train, y_train)

pred = model.predict(X_test)  # 테스트 데이터로 예측

print('accuracy:',accuracy_score(y_test, pred))
print('f1-score:',f1_score(y_test,pred))

[LightGBM] [Info] Number of positive: 956, number of negative: 3821
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006465 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1272
[LightGBM] [Info] Number of data points in the train set: 4777, number of used features: 32
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.200126 -> initscore=-1.385510
[LightGBM] [Info] Start training from score -1.385510
accuracy: 0.8426778242677824
f1-score: 0.5026455026455027


In [ ]:
print(confusion_matrix(y_test, pred))  # confusion matrix 출력

[[912  56]
 [132  95]]


In [ ]:
print(classification_report(y_test, pred))  # classification report 출력

              precision    recall  f1-score   support

           0       0.87      0.94      0.91       968
           1       0.63      0.42      0.50       227

    accuracy                           0.84      1195
   macro avg       0.75      0.68      0.70      1195
weighted avg       0.83      0.84      0.83      1195



### LightGBM Classifier에 있는 Hyperparameter
- LightGBMClassifier : https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html#lightgbm.LGBMClassifier
- 코드에서 활용한 하이퍼파라미터
> * objective: 모델의 학습 과제 및 학습목표(일반적으로 classifier의 경우 이진 혹은 다중 분류로 기본값이 설정되어 있음)
> * num_leaves: tree의 최대 leaf 수
> * learning_rate: 학습률
> * n_estimators: 학습에 사용할 트리 수
> * max_depth: 트리의 최대 깊이
> * ramdom_state: 결과 재현을 위한 시드값

- 아래 문구가 학습시 나오더라도 오류는 아니니 무시할 것(더 분할할 수 없어서 나타나는 경고)

  [Warning] No further splits with positive gain, best gain: -inf

In [ ]:
# Optuna
import optuna
def objective(trial):
    global X_train, X_test, y_train, y_test

    # 최적화 할 하이퍼파라미터 지정 및 값의 범위 설정
    param = {
        "objective": "binary",
        'num_leaves': trial.suggest_int('num_leaves', 2, 512, step=1, log=True),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 0.3),
        'n_estimators':  trial.suggest_int('n_estimators',100,1000 ),
        'max_depth': trial.suggest_int("max_depth", 3, 21, step=2),
        'random_state': 100,
    }

    model = lgb.LGBMRegressor(**param)
    lgb_model = model.fit(X_train, y_train, eval_set=[(X_test, y_test)])
    preds = lgb_model.predict(X_test)
    pred_labels = np.rint(preds)

    accuracy = accuracy_score(y_test, pred_labels)
    return accuracy


# optuna에서의 최적화 할 study 생성과 최적화 방향(지표의 값을 최대화 할 것인지)
study = optuna.create_study(direction="maximize")

# trial 횟수 설정 및 최적화 시작
study.optimize(objective, n_trials=100, timeout=600)

print("Number of finished trials: ", len(study.trials))

[I 2024-04-29 01:20:24,301] A new study created in memory with name: no-name-f92d2fce-005b-4cfe-8478-66014f93761e


[LightGBM] [Info] Number of positive: 956, number of negative: 3821
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005130 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1272
[LightGBM] [Info] Number of data points in the train set: 4777, number of used features: 32
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.200126 -> initscore=-1.385510
[LightGBM] [Info] Start training from score -1.385510


[I 2024-04-29 01:20:27,176] Trial 0 finished with value: 0.8451882845188284 and parameters: {'num_leaves': 32, 'learning_rate': 0.18261771034348856, 'n_estimators': 620, 'max_depth': 15}. Best is trial 0 with value: 0.8451882845188284.


[LightGBM] [Info] Number of positive: 956, number of negative: 3821
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000867 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1272
[LightGBM] [Info] Number of data points in the train set: 4777, number of used features: 32
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.200126 -> initscore=-1.385510
[LightGBM] [Info] Start training from score -1.385510


[I 2024-04-29 01:20:27,650] Trial 1 finished with value: 0.8284518828451883 and parameters: {'num_leaves': 4, 'learning_rate': 0.004825737780929112, 'n_estimators': 282, 'max_depth': 7}. Best is trial 0 with value: 0.8451882845188284.


...

[LightGBM] [Info] Number of positive: 956, number of negative: 3821
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000922 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1272
[LightGBM] [Info] Number of data points in the train set: 4777, number of used features: 32
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.200126 -> initscore=-1.385510
[LightGBM] [Info] Start training from score -1.385510
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

[I 2024-04-29 01:24:34,854] Trial 98 finished with value: 0.8485355648535565 and parameters: {'num_leaves': 330, 'learning_rate': 0.1081987034889802, 'n_estimators': 900, 'max_depth': 21}. Best is trial 13 with value: 0.8569037656903765.


[LightGBM] [Info] Number of positive: 956, number of negative: 3821
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000609 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1272
[LightGBM] [Info] Number of data points in the train set: 4777, number of used features: 32
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.200126 -> initscore=-1.385510
[LightGBM] [Info] Start training from score -1.385510
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

[I 2024-04-29 01:24:37,719] Trial 99 finished with value: 0.8426778242677824 and parameters: {'num_leaves': 453, 'learning_rate': 0.1911643535666991, 'n_estimators': 864, 'max_depth': 19}. Best is trial 13 with value: 0.8569037656903765.


Number of finished trials:  100
Best trial:


In [ ]:
# best trial에 대한 성능과 hyperparameter 정보 출력
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Best trial:
  Value: 0.8569037656903765
  Params: 
    num_leaves: 484
    learning_rate: 0.2250869675851037
    n_estimators: 999
    max_depth: 15


In [ ]:
#plot_optimization_history
optuna.visualization.plot_optimization_history(study)

In [ ]:
#plot_parallel_coordinate
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
#Visualize parameter importances.
optuna.visualization.plot_param_importances(study)